# Shear stress profile in uncracked region

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import bmcs_utils.api as bu
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import \
    DICGrid, DICInpUnstructuredPoints, DICStateFields, \
    DICAlignedGrid, DICGrid
from bmcs_shear.dic_crack.dic_crack_list2 import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');

Read the data and detect cracks

In [ ]:
dic_points = DICInpUnstructuredPoints(U_factor=10, dir_name='B7_TV1', t=1,
                                      n_T_max=40, T_stepping='delta_T')
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, d_x=5, d_y=5, t=0.2)
dsf = DICStateFields(dic_grid=dic_grid, R=6, omega_threshold=0.15, verbose_eval=True)
dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                      epsilon_f=0.0019, eps_max=0.01);
dsf = DICStateFields(dic_grid=dic_grid, R=8, omega_threshold=0.15, verbose_eval=True, tmodel='scalar_damage')
dsf.tmodel_.trait_set(E=1600, nu=0.18, omega_fn='exp-slope', strain_norm='Rankine', eps_max=0.01)
dsf.tmodel_.omega_fn_.trait_set(kappa_0=0.002, kappa_f=0.0028);
dcl = DICCrackList(dsf=dsf, 
                   t_detect=0.95,
                   delta_alpha_min=-0.5, # -np.pi/6, 
                   delta_alpha_max=np.pi/3, delta_s = 25, x_boundary=30)
dcl.interact()

## Stress profile in compressive region 

In [ ]:
c = 3
crack = dcl.cracks[c]
sp = crack.sp

In [ ]:
%matplotlib widget
fig, (ax_u, ax_F) = plt.subplots(1,2)
ax_sig = ax_F.twiny()
sp.plot_sig_t_crc_La(ax_sig)
sp.plot_sig_t_unc_Lab(ax_sig)
crack.plot_eps_unc_t_Kab(ax_u)
X_unc_t_La = sp.X_unc_t_La
ax_u.plot(sp.eps_unc_t_Lab[:, 0, 0], X_unc_t_La[:, 1], color='red')
sp.plot_F_t_a(ax_F)
bu.mpl_align_xaxis(ax_sig, ax_F)

In [ ]:
dcl.dsf.X_ipl_MNa.shape

In [ ]:
X_upper_line_Ma = dcl.dsf.X_ipl_MNa[:, -1, :]
x_M, y_M = X_upper_line_Ma.T
t_M = np.ones_like(x_M) * 0 # dic_grid.t
tX_Ma = np.array([t_M, x_M, y_M], dtype=np.float_).T

In [ ]:
eps_Mab = dcl.dsf.f_eps_fe_txy(tX_Ma)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
fig.canvas.header_visible = False
ax.plot(x_M, eps_Mab[:,0,0])

In [ ]:
from vtk.util import numpy_support

In [ ]:
#numpy_support.numpy_to_vtk(num_array=x_M.ravel(), deep=True, array_type=vtk.VTK_FLOAT)

## Direct quantification of compression stress from sthe strain profile 

In [ ]:
t = 1
sig_d_x = 250
sig2_d_x = sig_d_x / 2
x_K, y_K = crack.X_unc_t_Ka.T
t_K = np.ones_like(x_K) * t

tX_right_Ka = np.array([t_K, x_K + sig2_d_x, y_K], dtype=np.float_).T
tX_left_Ka = np.array([t_K, x_K - sig2_d_x, y_K], dtype=np.float_).T
ttX_right_Ka = np.array([x_K + sig2_d_x, y_K], dtype=np.float_).T
ttX_left_Ka = np.array([x_K - sig2_d_x, y_K], dtype=np.float_).T

In [ ]:
eps_left_Kab = crack.get_eps_Kab(crack.dic_grid.t, ttX_left_Ka)
eps_right_Kab = crack.get_eps_Kab(crack.dic_grid.t, ttX_right_Ka)

In [ ]:
eps_Kab = crack.get_eps_Kab(crack.dic_grid.t, crack.X_unc_t_Ka)

In [ ]:
%matplotlib widget
fig, ax_u = plt.subplots(1,1)
crack.plot_eps_unc_t_Kab(ax_u)
ax_u.plot(eps_left_Kab[:,0,0], y_K, color='red')
ax_u.plot(eps_right_Kab[:,0,0], y_K, color='green')
ax_u.plot(sp.eps_unc_t_Lab[:,0,0], X_unc_t_La[:, 1], color='orange');

## Stress profile

In [ ]:
E_, nu_ = 28000, 0.2

In [ ]:
_1_nu_ = 1 / (1 - nu_)
E_ab = E_ / (1 + nu_) * np.array([[ _1_nu_, _1_nu_, 0],
                                  [ _1_nu_, _1_nu_, 0],
                                  [ 0, 0, 0.5]])

In [ ]:
eps_left_Kd = np.hstack([eps_left_Kab[:, [0, 1], [0, 1]], 2 * eps_left_Kab[:, 0, 1, np.newaxis]])
eps_right_Kd = np.hstack([eps_right_Kab[:, [0, 1], [0, 1]], 2 * eps_right_Kab[:, 0, 1, np.newaxis]])

In [ ]:
sig_left_Kd = np.einsum('cd, ...d->...c', E_ab, eps_left_Kd)
sig_right_Kd = np.einsum('cd, ...d->...c', E_ab, eps_right_Kd)

In [ ]:
delta_sig_Kd = sig_right_Kd - sig_left_Kd

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
fig.canvas.header_visible = False
eps_K_ = sig_right_Kd[:, 0]
ax.plot(eps_K_, y_K)
sp.plot_sig_t_unc_Lab(ax)
ax.plot(np.zeros_like(eps_K_), y_K)